# Conversion Tools
This a notebook containing various scripts to convert things

## 1 - convert a pytorch model to something I can use in keras

In [19]:
import torch
import torchvision.models as models
import onnx
from onnx2keras import onnx_to_keras

In [18]:
fer_model = models.resnet18(num_classes=7)

checkpoint = torch.load('../papers/Emotion-FAN/pretrain_model/Resnet18_FER+_pytorch.pth.tar', map_location=torch.device('cpu'))
pretrained_state_dict = checkpoint['state_dict']
model_state_dict = fer_model.state_dict()

for key in pretrained_state_dict:
    if ((key == 'module.fc.weight') | (key == 'module.fc.bias')):
        pass
    else:
        model_state_dict[key.replace('module.', '')] = pretrained_state_dict[key]

fer_model.load_state_dict(model_state_dict)
model = torch.nn.DataParallel(fer_model)

model.eval()

x = torch.randn(64, 3, 244, 244, requires_grad=True)

torch.onnx.export(model.module, x, "../models/fan-fer.onnx")



In [24]:
onnx_model = onnx.load("../models/fan-fer.onnx")
onnx.checker.check_model(onnx_model)
k_model = onnx_to_keras(onnx_model, ['input.1'])
k_model.save('../models/fan-fer')

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.1.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> 191.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight fc.weight with shape (7, 512).
DEBUG:onnx2keras:Found weight fc.bias with shape (7,).
DEBUG:onnx2keras:Found weight 193 with shape (64, 3, 7, 7).
DEBUG:onnx2keras:Found weight 194 with shape (64,).
DEBUG:onnx2keras:Found weight 196 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found weight 197 with shape (64,).
DEBUG:onnx2keras:Found weight 199 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found weight 200 with shape (64,).
DEBUG:onnx2keras:Found weight 202 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found weight 203 with shape (64,).
DEBUG:onnx2keras:Found weight 205 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found weight 206 with shape (64,).
DEBUG:onnx2keras:Found weight 208 with shap

['input.1']


DEBUG:onnx2keras:Check input 0 (name 136).
DEBUG:onnx2keras:Check input 1 (name 205).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 206).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:Output TF Layer -> Tensor("204_1/Identity:0", shape=(None, 64, 61, 61), dtype=float32)
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Add
DEBUG:onnx2keras:node_name: 139
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 204).
DEBUG

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ../models/fan-fer/assets


INFO:tensorflow:Assets written to: ../models/fan-fer/assets


In [17]:

ms1m_model = models.resnet18(num_classes=7)

checkpoint = torch.load('../papers/Emotion-FAN/pretrain_model/Resnet18_MS1M_pytorch.pth.tar', map_location=torch.device('cpu'))
pretrained_state_dict = checkpoint['state_dict']
model_state_dict = ms1m_model.state_dict()

for key in pretrained_state_dict:
    if ((key == 'module.fc.weight') | (key == 'module.fc.bias')):
        pass
    else:
        model_state_dict[key.replace('module.', '')] = pretrained_state_dict[key]

ms1m_model.load_state_dict(model_state_dict)
model = torch.nn.DataParallel(ms1m_model)

model.eval()

x = torch.randn(64, 3, 244, 244, requires_grad=True)

# if using MS1M, then fix this which goes wrong for some reason
torch.onnx.export(model.module, x, "../models/fan-ms1m.onnx")


RuntimeError: Error(s) in loading state_dict for ResNet:
	Unexpected key(s) in state_dict: "feature.weight", "feature.bias". 